<a href="https://colab.research.google.com/github/kumar6rishabh/resnet_from_scratch/blob/main/resnet18_architecture.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [ ]:
def get_residual_block(num_fils , first_block = False , first_layer = False):
  ret_stride = 2 if first_block else 1
  in_fils = int(num_fils / 2) if first_block else num_fils
  if first_block and first_layer:
    ret_stride = 1
    in_fils = num_fils
  block = nn.Sequential(nn.Conv2d(in_fils , num_fils , kernel_size = 3 , stride = ret_stride , padding = 1) , 
  nn.ReLU(inplace = True) , 
  nn.Conv2d(num_fils, num_fils , kernel_size = 3 , stride = 1 , padding = 1))
  return block

In [ ]:
class Net(nn.Module):
  def __init__(self):
    super(Net , self).__init__()
    self.initial_conv = nn.Conv2d(3 , 64 , kernel_size = 7 , stride = 2 , padding = 1)
    self.relu = nn.ReLU(inplace = True)
    self.initial_pool = nn.MaxPool2d(kernel_size = 2)
    self.avgpool = nn.AvgPool2d(kernel_size = (7 , 7))
    self.fc = nn.Linear(512 , 1000)
  def forward(self , x):
    x = self.initial_conv(x)
    x = self.relu(x)
    x = self.initial_pool(x)
    x = self.make_layer(x , 3 , 64 , True)
    x = self.make_layer(x , 4 , 128)
    x = self.make_layer(x , 6 , 256)
    x = self.make_layer(x , 3 , 512)
    x = self.avgpool(x)
    x = torch.flatten(x , 1)
    x = self.fc(x)
    return x

    
  def make_layer(self , inp , num_blocks , num_fils , first_layer = False):
    back = inp
    if first_layer:
      x = get_residual_block(num_fils , True , True)(inp)
    else:
      back = nn.Conv2d(int(num_fils / 2) , num_fils , kernel_size = 1 , stride = 2 , padding = 0)(inp)
      x = get_residual_block(num_fils , True)(inp)
    back += x #skip connection
    back = F.relu(back)
    for _ in range(num_blocks - 1):
      x = get_residual_block(num_fils)(back)
      back += x
      back = F.relu(back)
    return back

In [ ]:
model = Net()

In [ ]:
img = torch.randn(5 , 3 , 224 , 224)

In [ ]:
rt = model(img)

In [ ]:
rt.shape

torch.Size([5, 1000])

In [ ]:
model

Net(
  (initial_conv): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(1, 1))
  (relu): ReLU(inplace=True)
  (initial_pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (avgpool): AvgPool2d(kernel_size=(7, 7), stride=(7, 7), padding=0)
  (fc): Linear(in_features=512, out_features=1000, bias=True)
)

In [ ]:
nn.Conv2d(3 , 64 , kernel_size = 7 , stride = 2 , padding = 1)(img).shape

torch.Size([1, 64, 110, 110])